# Inteligencia Artificial - ESIS - UNJBG
## Semana 2: Entorno, Tensores, CUDA
### Docente: MSc.(c) Israel N. Chaparro-Cruz
**Basado en:** Week 1, Day 1: Basics and PyTorch, By Neuromatch Academy

https://academy.neuromatch.io/

https://github.com/NeuromatchAcademy/course-content-dl

__Content creators:__ Shubh Pachchigar, Vladimir Haltakov, Matthew Sargent, Konrad Kording

__Content reviewers:__ Deepak Raya, Siwei Bai, Kelson Shilling-Scrivo

__Content editors:__ Anoop Kulkarni, Spiros Chavlis

__Production editors:__ Arush Tagade, Spiros Chavlis

__Post-Production team:__ Gagana B, Spiros Chavlis

__Content Traduction:__ Israel N. Chaparro-Cruz

---
# Objetivos del tutorial

A continuación, tenemos algunos objetivos específicos para este tutorial:
* Aprender sobre PyTorch y los tensores
* Manipulaciones de tensores
* Carga de datos
* GPUs y Cuda Tensors
* Entrenar NaiveNet

---
# Sección 1: Configuración

In [1]:
# Imports
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch libraries
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
# @title Figure Settings
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/content-creation/main/nma.mplstyle")

In [3]:
# @title Helper Functions

def checkExercise1(A, B, C, D):
  """
  Helper function for checking Exercise 1.

  Args:
    A: torch.Tensor
      Torch Tensor of shape (20, 21) consisting of ones.
    B: torch.Tensor
      Torch Tensor of size([3,4])
    C: torch.Tensor
      Torch Tensor of size([20,21])
    D: torch.Tensor
      Torch Tensor of size([19])

  Returns:
    Nothing.
  """
  assert torch.equal(A.to(int),torch.ones(20, 21).to(int)), "Got: {A} \n Expected: {torch.ones(20, 21)} (shape: {torch.ones(20, 21).shape})"
  assert np.array_equal(B.numpy(),np.vander([1, 2, 3], 4)), "Got: {B} \n Expected: {np.vander([1, 2, 3], 4)} (shape: {np.vander([1, 2, 3], 4).shape})"
  assert C.shape == (20, 21), "Got: {C} \n Expected (shape: {(20, 21)})"
  assert torch.equal(D, torch.arange(4, 41, step=2)), "Got {D} \n Expected: {torch.arange(4, 41, step=2)} (shape: {torch.arange(4, 41, step=2).shape})"
  print("All correct")

def timeFun(f, dim, iterations, device='cpu'):
  """
  Helper function to calculate amount of time taken per instance on CPU/GPU

  Args:
    f: BufferedReader IO instance
      Function name for which to calculate computational time complexity
    dim: Integer
      Number of dimensions in instance in question
    iterations: Integer
      Number of iterations for instance in question
    device: String
      Device on which respective computation is to be run

  Returns:
    Nothing
  """
  iterations = iterations
  t_total = 0
  for _ in range(iterations):
    start = time.time()
    f(dim, device)
    end = time.time()
    t_total += end - start

  if device == 'cpu':
    print(f"time taken for {iterations} iterations of {f.__name__}({dim}, {device}): {t_total:.5f}")
  else:
    print(f"time taken for {iterations} iterations of {f.__name__}({dim}, {device}): {t_total:.5f}")

---
# Sección 2: Fundamentos de Pytorch

PyTorch es un paquete de computación científica basado en Python y dirigido a dos grupos de
públicos:

- Un sustituto de NumPy optimizado para la potencia de las GPU
- Una plataforma de aprendizaje profundo que proporciona una gran flexibilidad
   y velocidad

En su núcleo, PyTorch proporciona algunas características clave:

- Un objeto multidimensional [Tensor](https://pytorch.org/docs/stable/tensors.html), similar a [NumPy Array](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) pero con aceleración en la GPU.
- Un motor **autograd** optimizado para calcular automáticamente las derivadas.
- Una API limpia y modular para construir y desplegar **modelos de aprendizaje profundo**.

Puedes encontrar más información sobre PyTorch en el Apéndice.

## Sección 2.1: Creando Tensores


Hay varias formas de crear tensores, y al realizar cualquier proyecto real de aprendizaje profundo, normalmente tendremos que hacerlo.

**Construir tensores directamente:**

---

In [ ]:
# We can construct a tensor directly from some common python iterables,
# such as list and tuple nested iterables can also be handled as long as the
# dimensions are compatible

# tensor from a list
a = torch.tensor([0, 1, 2])

#tensor from a tuple of tuples
b = ((1.0, 1.1), (1.2, 1.3))
b = torch.tensor(b)

# tensor from a numpy array
c = np.ones([2, 3])
c = torch.tensor(c)

print(f"Tensor a: {a}")
print(f"Tensor b: {b}")
print(f"Tensor c: {c}")

**Algunos constructores de tensor comunes:**

---

In [ ]:
# The numerical arguments we pass to these constructors
# determine the shape of the output tensor

x = torch.ones(5, 3)
y = torch.zeros(2)
z = torch.empty(1, 1, 5)
print(f"Tensor x: {x}")
print(f"Tensor y: {y}")
print(f"Tensor z: {z}")

Observe que `.empty()` no devuelve ceros, sino números aparentemente aleatorios. A diferencia de `.zeros()`, que inicializa los elementos del tensor con ceros, `.empty()` sólo asigna la memoria. Por lo tanto, es un poco más rápido si lo que quieres es crear un tensor.

**Crear tensores aleatorios y tensores como otros tensores:**

---

In [ ]:
# There are also constructors for random numbers

# Uniform distribution
a = torch.rand(1, 3)

# Normal distribution
b = torch.randn(3, 4)

# There are also constructors that allow us to construct
# a tensor according to the above constructors, but with
# dimensions equal to another tensor.

c = torch.zeros_like(a)
d = torch.rand_like(c)

print(f"Tensor a: {a}")
print(f"Tensor b: {b}")
print(f"Tensor c: {c}")
print(f"Tensor d: {d}")

*Reproductibilidad*: 

- Generador de números aleatorios (RNG) de PyTorch: Puedes usar `torch.manual_seed()` para sembrar el RNG para todos los dispositivos (tanto CPU como GPU):

```python
import torch
torch.manual_seed(0)
```
- Para los operadores personalizados, es posible que tenga que establecer python seed también:

```python
import random
random.seed(0)
```

- Generadores de números aleatorios en otras bibliotecas (por ejemplo, NumPy):

```python
import numpy as np
np.random.seed(0)
```

Aquí, definimos para ti una función llamada `set_seed` que hace el trabajo por ti.

In [7]:
def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')

Ahora, vamos a utilizar la función `set_seed` del ejemplo anterior. Ejecute la celda varias veces para verificar que los números impresos son siempre los mismos.

In [8]:
def simplefun(seed=True, my_seed=None):
  """
  Helper function to verify effectiveness of set_seed attribute

  Args:
    seed: Boolean
      Specifies if seed value is provided or not
    my_seed: Integer
      Initializes seed to specified value

  Returns:
    Nothing
  """
  if seed:
    set_seed(seed=my_seed)

  # uniform distribution
  a = torch.rand(1, 3)
  # normal distribution
  b = torch.randn(3, 4)

  print("Tensor a: ", a)
  print("Tensor b: ", b)

In [ ]:
simplefun(seed=True, my_seed=0)  # Turn `seed` to `False` or change `my_seed`

**Rangos de números tipo Numpy:**
---
Las funciones ``.arange()`` y ``.linspace()`` se comportan como es de esperar si estás familiarizado con numpy.

In [ ]:
a = torch.arange(0, 10, step=1)
b = np.arange(0, 10, step=1)

c = torch.linspace(0, 5, steps=11)
d = np.linspace(0, 5, num=11)

print(f"Tensor a: {a}\n")
print(f"Numpy array b: {b}\n")
print(f"Tensor c: {c}\n")
print(f"Numpy array d: {d}\n")

### Ejercicio 2.1: Creación de Tensores

A continuación encontrarás un código incompleto. Completa el código que falta para construir los tensores especificados.

Queremos los tensores 

$A:$ un tensor de 20 x 21 formado por unos

$B:$ un tensor con elementos iguales a los elementos del array $Z$ de numpy

$C:$ un tensor con el mismo número de elementos que $A$ pero con valores $
\sim \mathcal{U}(0,1)^\dagger$

$D:$ un tensor 1D que contiene los números pares entre 4 y 40 inclusive.

<br>

$^\dagger$: $\mathcal{U(\alpha, \beta)}$ denota la [distribución uniforme](https://en.wikipedia.org/wiki/Continuous_uniform_distribution) de $\alpha$ a $\beta$, con $\alpha, \beta \in \mathbb{R}$.



In [19]:
def tensor_creation(Z):
  """
  A function that creates various tensors.

  Args:
    Z: numpy.ndarray
      An array of shape (3,4)

  Returns:
    A : Tensor
      20 by 21 tensor consisting of ones
    B : Tensor
      A tensor with elements equal to the elements of numpy array Z
    C : Tensor
      A tensor with the same number of elements as A but with values ∼U(0,1)
    D : Tensor
      A 1D tensor containing the even numbers between 4 and 40 inclusive.
  """
  #################################################
  ## TODO for students: fill in the missing code
  ## from the first expression
  raise NotImplementedError("Student exercise: say what they should have done")
  #################################################
  A = ...
  B = ...
  C = ...
  D = ...

  return A, B, C, D

# numpy array to copy later
#Z = np.vander([1, 2, 3], 4)

# Uncomment below to check your function!
#A, B, C, D = tensor_creation(Z)
#checkExercise1(A, B, C, D)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_231162bb.py)



```
All correct!
```

## Sección 2.2: Operaciones en PyTorch

**Operaciones con tensores**

Podemos realizar operaciones sobre tensores utilizando métodos en `torch.`

In [ ]:
a = torch.ones(5, 3)
b = torch.rand(5, 3)
c = torch.empty(5, 3)
d = torch.empty(5, 3)

# this only works if c and d already exist
torch.add(a, b, out=c)

# Pointwise Multiplication of a and b
torch.multiply(a, b, out=d)

print(c)
print(d)

Sin embargo, en PyTorch, la mayoría de los operadores comunes de Python están anulados.

Los operadores aritméticos estándar comunes ($+$, $-$, $*$, $/$ y $**$) han sido elevados a operaciones elementales

In [ ]:
x = torch.tensor([1, 2, 4, 8])
y = torch.tensor([1, 2, 3, 4])
x + y, x - y, x * y, x / y, x**y  # The `**` is the exponentiation operator

**Métodos Tensoriales**

Los tensores también tienen un número de operaciones aritméticas comunes incorporadas. Una lista completa de **todos** los métodos se puede encontrar en el apéndice (¡hay muchos!) 

Todas estas operaciones deberían tener una sintaxis similar a la de sus equivalentes en numpy (¡siéntete libre de omitirlas si ya las conoces!).

In [ ]:
x = torch.rand(3, 3)
print(x)
print("\n")
# sum() - note the axis is the axis you move across when summing
print(f"Sum of every element of x: {x.sum()}")
print(f"Sum of the columns of x: {x.sum(axis=0)}")
print(f"Sum of the rows of x: {x.sum(axis=1)}")
print("\n")

print(f"Mean value of all elements of x {x.mean()}")
print(f"Mean values of the columns of x {x.mean(axis=0)}")
print(f"Mean values of the rows of x {x.mean(axis=1)}")

**Operaciones matriciales**

El símbolo `@` se anula para representar la multiplicación de matrices. También puede utilizar `torch.matmul()` para multiplicar tensores. Para la multiplicación por puntos, puede utilizar `torch.dot()`, o manipular los ejes de sus tensores y hacer la multiplicación matricial (lo veremos en la siguiente sección). 

Las transposiciones de los tensores 2D se obtienen utilizando `torch.t()` o `Tensor.T`. Observe la falta de paréntesis para `Tensor.T` - es un atributo, no un método.



### Ejercicio 2.2 : Operaciones tensoriales simples

A continuación se muestran dos expresiones que implican operaciones con matrices. 

\begin{equation}
\textbf{A} = 
\begin{bmatrix}2 &4 \\5 & 7 
\end{bmatrix} 
\begin{bmatrix} 1 &1 \\2 & 3
\end{bmatrix} 
+ 
\begin{bmatrix}10 & 10  \\ 12 & 1 
\end{bmatrix} 
\end{equation}


y


\begin{equation}
b = 
\begin{bmatrix} 3 \\ 5 \\ 7
\end{bmatrix} \cdot 
\begin{bmatrix} 2 \\ 4 \\ 8
\end{bmatrix}
\end{equation}

El bloque de código de abajo que calcula estas expresiones usando PyTorch está incompleto - complete las líneas que faltan.

In [24]:
def simple_operations(a1: torch.Tensor, a2: torch.Tensor, a3: torch.Tensor):
  """
  Helper function to demonstrate simple operations
  i.e., Multiplication of tensor a1 with tensor a2 and then add it with tensor a3

  Args:
    a1: Torch tensor
      Tensor of size ([2,2])
    a2: Torch tensor
      Tensor of size ([2,2])
    a3: Torch tensor
      Tensor of size ([2,2])

  Returns:
    answer: Torch tensor
      Tensor of size ([2,2]) resulting from a1 multiplied with a2, added with a3
  """
  ################################################
  ## TODO for students:  complete the first computation using the argument matricies
  raise NotImplementedError("Student exercise: fill in the missing code to complete the operation")
  ################################################
  #
  answer = ...
  return answer

# Computing expression 1:

# init our tensors
a1 = torch.tensor([[2, 4], [5, 7]])
a2 = torch.tensor([[1, 1], [2, 3]])
a3 = torch.tensor([[10, 10], [12, 1]])
## uncomment to test your function
#A = simple_operations(a1, a2, a3)
#print(A)

In [ ]:
def dot_product(b1: torch.Tensor, b2: torch.Tensor):
  ###############################################
  ## TODO for students:  complete the first computation using the argument matricies
  raise NotImplementedError("Student exercise: fill in the missing code to complete the operation")
  ###############################################
  """
  Helper function to demonstrate dot product operation
  Dot product is an algebraic operation that takes two equal-length sequences
  (usually coordinate vectors), and returns a single number.
  Geometrically, it is the product of the Euclidean magnitudes of the
  two vectors and the cosine of the angle between them.

  Args:
    b1: Torch tensor
      Tensor of size ([3])
    b2: Torch tensor
      Tensor of size ([3])

  Returns:
    product: Tensor
      Tensor of size ([1]) resulting from b1 scalar multiplied with b2
  """
  # Use torch.dot() to compute the dot product of two tensors
  product = ...
  return product

# Computing expression 2:
b1 = torch.tensor([3, 5, 7])
b2 = torch.tensor([2, 4, 8])
## Uncomment to test your function
# b = dot_product(b1, b2)
# print(b)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_d43f1ed4.py)



```
tensor([[20, 24],
        [31, 27]])
```

## Sección 2.3 Manipulación de tensores en Pytorch

**Indexación**

Al igual que en numpy, se puede acceder a los elementos de un tensor por medio de un índice. Como en cualquier array de numpy, el primer elemento tiene el índice 0 y los rangos se especifican para incluir del primero al último_elemento-1. Podemos acceder a los elementos según su posición relativa al final de la lista utilizando índices negativos. La indexación también se denomina rebanada.

Por ejemplo, `[-1]` selecciona el último elemento; `[1:3]` selecciona el segundo y el tercer elemento, y `[:-2]` seleccionará todos los elementos excluyendo el último y el penúltimo.

In [ ]:
x = torch.arange(0, 10)
print(x)
print(x[-1])
print(x[1:3])
print(x[:-2])

Cuando tenemos tensores multidimensionales, las reglas de indexación funcionan igual que en NumPy.

In [ ]:
# make a 5D tensor
x = torch.rand(1, 2, 3, 4, 5)

print(f" shape of x[0]:{x[0].shape}")
print(f" shape of x[0][0]:{x[0][0].shape}")
print(f" shape of x[0][0][0]:{x[0][0][0].shape}")

**Aplanar y remodelar**

Existen varios métodos para remodelar tensores. Es común tener que expresar datos 2D en formato 1D. Del mismo modo, también es común tener que remodelar un tensor 1D en un tensor 2D. Podemos conseguirlo con los métodos `.flatten()` y `.reshape()`.

In [ ]:
z = torch.arange(12).reshape(6, 2)
print(f"Original z: \n {z}")

# 2D -> 1D
z = z.flatten()
print(f"Flattened z: \n {z}")

# and back to 2D
z = z.reshape(3, 4)
print(f"Reshaped (3x4) z: \n {z}")

También verás que los métodos `.view()` se utilizan mucho para remodelar tensores. Hay una sutil diferencia entre `.view()` y `.reshape()`, aunque por ahora sólo usaremos `.reshape()`. La documentación se puede encontrar en el Apéndice.

**Exprimir los tensores**

Cuando se procesan lotes de datos, a menudo se encuentran dimensiones únicas. Por ejemplo, `[1,10]` o `[256, 1, 3]`. Esta dimensión puede fácilmente estropear tus operaciones matriciales si no planeas que esté ahí...

Para comprimir los tensores a lo largo de sus dimensiones simples podemos utilizar el método `.squeeze()`. Podemos utilizar el método `.unsqueeze()` para hacer lo contrario.

In [ ]:
x = torch.randn(1, 10)
# printing the zeroth element of the tensor will not give us the first number!

print(x.shape)
print(f"x[0]: {x[0]}")

Debido a esa molesta dimensión singleton, `x[0]` nos dio la primera fila en su lugar.

In [ ]:
# Let's get rid of that singleton dimension and see what happens now
x = x.squeeze(0)
print(x.shape)
print(f"x[0]: {x[0]}")

In [ ]:
# Adding singleton dimensions works a similar way, and is often used when tensors
# being added need same number of dimensions

y = torch.randn(5, 5)
print(f"Shape of y: {y.shape}")

# lets insert a singleton dimension
y = y.unsqueeze(1)
print(f"Shape of y: {y.shape}")

**Permutación**

A veces nuestras dimensiones estarán en el orden equivocado. Por ejemplo, podemos estar tratando con imágenes RGB con dim $[3\times48\times64]$, pero nuestro pipeline espera que la dimensión de color sea la última dimensión, es decir, $[48\times64\times3]$. Para evitar esto podemos utilizar el método `.permute()`.

In [ ]:
# `x` has dimensions [color,image_height,image_width]
x = torch.rand(3, 48, 64)

# We want to permute our tensor to be [ image_height , image_width , color ]
x = x.permute(1, 2, 0)
# permute(1,2,0) means:
# The 0th dim of my new tensor = the 1st dim of my old tensor
# The 1st dim of my new tensor = the 2nd
# The 2nd dim of my new tensor = the 0th
print(x.shape)

También puede ver el uso de `.transpose()`. Esto funciona de forma similar a permutar, pero sólo puede intercambiar dos dimensiones a la vez.

**Concatenación**

En este ejemplo, concatenamos dos matrices a lo largo de las filas (eje 0, el primer elemento de la forma) frente a las columnas (eje 1, el segundo elemento de la forma). Podemos ver que la longitud del eje 0 del primer tensor de salida (`6`) es la suma de las longitudes del eje 0 de los dos tensores de entrada (`3+3`); mientras que la longitud del eje 1 del segundo tensor de salida (`8`) es la suma de las longitudes del eje 1 de los dos tensores de entrada (`4+4`).

In [ ]:
# Create two tensors of the same shape
x = torch.arange(12, dtype=torch.float32).reshape((3, 4))
y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

# Concatenate along rows
cat_rows = torch.cat((x, y), dim=0)

# Concatenate along columns
cat_cols = torch.cat((x, y), dim=1)

# Printing outputs
print('Concatenated by rows: shape{} \n {}'.format(list(cat_rows.shape), cat_rows))
print('\n Concatenated by colums: shape{}  \n {}'.format(list(cat_cols.shape), cat_cols))

**Conversión a otros objetos de Python**

Convertir un tensor a un numpy.ndarray, o viceversa, es fácil, y el resultado convertido no comparte memoria. Este pequeño inconveniente es bastante importante: cuando realizas operaciones en la CPU o en la GPU, no quieres detener el cálculo, esperando a ver si el paquete NumPy de Python puede querer estar haciendo otra cosa con el mismo trozo de memoria.

Al convertir a un array de NumPy, se perderá la información que está siendo rastreada por el tensor, es decir, el grafo computacional.

In [ ]:
x = torch.randn(5)
print(f"x: {x}  |  x type:  {x.type()}")

y = x.numpy()
print(f"y: {y}  |  y type:  {type(y)}")

z = torch.tensor(y)
print(f"z: {z}  |  z type:  {z.type()}")

Para convertir un tensor de tamaño 1 en un escalar de Python, podemos invocar la función item o las funciones incorporadas de Python.

In [ ]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

### Ejercicio 2.3: Manipulación de tensores
Usando una combinación de los métodos discutidos anteriormente, completa las funciones de abajo.

**Function A** 

**Función A** 

Esta función toma dos tensores 2D $A$ y $B$ y devuelve la suma de columnas de A multiplicada por la suma de todos los elmementos de $B$, es decir, un escalar, por ejemplo,

\begin{equation}
  \text{If }
  A = \begin{bmatrix}
  1 & 1 \\
  1 & 1
  \end{bmatrix}
  \text{and }
  B = \begin{bmatrix}
  1 & 2 & 3 \\
  1 & 2 & 3
  \end{bmatrix}
  \text{ then }
  Out =  \begin{bmatrix}
  2 & 2
  \end{bmatrix} \cdot 12 = \begin{bmatrix}
  24 & 24
  \end{bmatrix}
\end{equation}

**Función B** 

Esta función toma una matriz cuadrada $C$ y devuelve un tensor 2D que consiste en una $C$ aplanada con el índice de cada elemento añadido a este tensor en la dimensión de la fila, por ejemplo,

\begin{equation}
  \text{If }
  C = \begin{bmatrix}
  2 & 3 \\
  -1 & 10
  \end{bmatrix}
  \text{ then }
  Out = \begin{bmatrix}
  0 & 2 \\
  1 & 3 \\
  2 & -1 \\
  3 & 10
  \end{bmatrix}
\end{equation}

**Sugerencia:** Preste mucha atención a las dimensiones únicas.

**Función C**

Esta función toma dos tensores 2D $D$ y $E$. Si las dimensiones lo permiten, esta función devuelve la suma elemental de $D$ en forma de $E$, y $D$; si no, esta función devuelve un tensor 1D que es la concatenación de los dos tensores, por ejemplo,

\begin{equation}
  \text{If }
  D = \begin{bmatrix}
  1 & -1 \\
  -1 & 3
  \end{bmatrix}
  \text{and } 
  E = \begin{bmatrix}
  2 & 3 & 0 & 2 \\
  \end{bmatrix}
  \text{ then } 
  Out = \begin{bmatrix}
  3 & 2 \\
  -1 & 5
  \end{bmatrix}
\end{equation}

<br>

\begin{equation}
  \text{If }
  D = \begin{bmatrix}
  1 & -1 \\
  -1 & 3
  \end{bmatrix}
  \text{and }
  E = \begin{bmatrix}
  2 & 3 & 0  \\
  \end{bmatrix}
  \text{ then }
  Out = \begin{bmatrix}
  1 & -1 & -1 & 3  & 2 & 3 & 0  
  \end{bmatrix}
\end{equation}

<br>

**Sugerencia:** `torch.numel()` es una manera fácil de encontrar el número de elementos en un tensor.

In [ ]:
def functionA(my_tensor1, my_tensor2):
  """
  This function takes in two 2D tensors `my_tensor1` and `my_tensor2`
  and returns the column sum of
  `my_tensor1` multiplied by the sum of all the elmements of `my_tensor2`,
  i.e., a scalar.

  Args:
    my_tensor1: torch.Tensor
    my_tensor2: torch.Tensor

  Retuns:
    output: torch.Tensor
      The multiplication of the column sum of `my_tensor1` by the sum of
      `my_tensor2`.
  """
  ################################################
  ## TODO for students: complete functionA
  raise NotImplementedError("Student exercise: complete function A")
  ################################################
  # TODO multiplication the sum of the tensors
  output = ...

  return output


def functionB(my_tensor):
  """
  This function takes in a square matrix `my_tensor` and returns a 2D tensor
  consisting of a flattened `my_tensor` with the index of each element
  appended to this tensor in the row dimension.

  Args:
    my_tensor: torch.Tensor

  Returns:
    output: torch.Tensor
      Concatenated tensor.
  """
  ################################################
  ## TODO for students: complete functionB
  raise NotImplementedError("Student exercise: complete function B")
  ################################################
  # TODO flatten the tensor `my_tensor`
  my_tensor = ...
  # TODO create the idx tensor to be concatenated to `my_tensor`
  idx_tensor = ...
  # TODO concatenate the two tensors
  output = ...

  return output


def functionC(my_tensor1, my_tensor2):
  """
  This function takes in two 2D tensors `my_tensor1` and `my_tensor2`.
  If the dimensions allow it, it returns the
  elementwise sum of `my_tensor1`-shaped `my_tensor2`, and `my_tensor2`;
  else this function returns a 1D tensor that is the concatenation of the
  two tensors.

  Args:
    my_tensor1: torch.Tensor
    my_tensor2: torch.Tensor

  Returns:
    output: torch.Tensor
      Concatenated tensor.
  """
  ################################################
  ## TODO for students: complete functionB
  raise NotImplementedError("Student exercise: complete function C")
  ################################################
  # TODO check we can reshape `my_tensor2` into the shape of `my_tensor1`
  if ...:
    # TODO reshape `my_tensor2` into the shape of `my_tensor1`
    my_tensor2 = ...
    # TODO sum the two tensors
    output = ...
  else:
    # TODO flatten both tensors
    my_tensor1 = ...
    my_tensor2 = ...
    # TODO concatenate the two tensors in the correct dimension
    output = ...

  return output

## Implement the functions above and then uncomment the following lines to test your code
# print(functionA(torch.tensor([[1, 1], [1, 1]]), torch.tensor([[1, 2, 3], [1, 2, 3]])))
# print(functionB(torch.tensor([[2, 3], [-1, 10]])))
# print(functionC(torch.tensor([[1, -1], [-1, 3]]), torch.tensor([[2, 3, 0, 2]])))
# print(functionC(torch.tensor([[1, -1], [-1, 3]]), torch.tensor([[2, 3, 0]])))

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_74d86edd.py)



```
tensor([24, 24])
tensor([[ 0,  2],
        [ 1,  3],
        [ 2, -1],
        [ 3, 10]])
tensor([[ 3,  2],
        [-1,  5]])
tensor([ 1, -1, -1,  3,  2,  3,  0])
```

## Sección 2.4: GPUs

¡Por defecto, cuando creamos un tensor éste *no* vivirá en la GPU! 

In [ ]:
x = torch.randn(10)
print(x.device)

Al utilizar los Colab notebook, por defecto, no tendremos acceso a una GPU. Para empezar a utilizar las GPUs tenemos que solicitar una. Podemos hacerlo yendo a la pestaña de tiempo de ejecución en la parte superior de la página. 

Siguiendo *Runtime* → *Cambiar tipo de runtime* y seleccionando **GPU** de la lista desplegable *Acelerador de hardware*, podemos empezar a jugar con el envío de tensores a las GPU.

Una vez hecho esto tu runtime se reiniciará y tendrás que volver a ejecutar la primera celda de configuración para reimportar PyTorch. Después, pasa a la siguiente celda.

Para más información sobre la política de uso de la GPU puedes ver en el Apéndice.

**Ahora tenemos una GPU.**

La celda de abajo debería devolver `True`.

In [ ]:
print(torch.cuda.is_available())

[CUDA](https://developer.nvidia.com/cuda-toolkit) es una API desarrollada por Nvidia para interactuar con las GPUs. PyTorch nos proporciona una capa de abstracción, y nos permite lanzar kernels CUDA utilizando Python puro.

En resumen, obtenemos la potencia de paralelizar nuestros cálculos tensoriales en las GPUs, ¡mientras sólo escribimos un Python (relativamente) sencillo!

Aquí definimos la función `set_device`, que devuelve el uso del dispositivo en el cuaderno, es decir, `cpu` o `cuda`. A menos que se especifique lo contrario, utilizamos esta función en la parte superior de cada tutorial, y almacenamos la variable de dispositivo como

``python
DEVICE = set_device()
```

Vamos a definir la función utilizando el paquete PyTorch `torch.cuda`, que se inicializa de forma perezosa, por lo que siempre podemos importarlo, y utilizar `is_available()` para determinar si nuestro sistema soporta CUDA.

In [14]:
def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("GPU is not enabled in this notebook. \n"
          "If you want to enable it, in the menu under `Runtime` -> \n"
          "`Hardware accelerator.` and select `GPU` from the dropdown menu")
  else:
    print("GPU is enabled in this notebook. \n"
          "If you want to disable it, in the menu under `Runtime` -> \n"
          "`Hardware accelerator.` and select `None` from the dropdown menu")

  return device

¡Hagamos algunos tensores CUDA!

In [ ]:
# common device agnostic way of writing code that can run on cpu OR gpu
# that we provide for you in each of the tutorials
DEVICE = set_device()

# we can specify a device when we first create our tensor
x = torch.randn(2, 2, device=DEVICE)
print(x.dtype)
print(x.device)

# we can also use the .to() method to change the device a tensor lives on
y = torch.randn(2, 2)
print(f"y before calling to() | device: {y.device} | dtype: {y.type()}")

y = y.to(DEVICE)
print(f"y after calling to() | device: {y.device} | dtype: {y.type()}")

**Operaciones entre tensores cpu y tensores cuda**

Observa que el tipo del tensor ha cambiado después de llamar a `.to()`. Qué ocurre si intentamos realizar operaciones con tensores en los dispositivos?

In [36]:
x = torch.tensor([0, 1, 2], device=DEVICE)
y = torch.tensor([3, 4, 5], device="cpu")

## Uncomment the following line and run this cell
#z = x + y

No podemos combinar los tensores de CUDA y los de la CPU de esta manera. Si queremos calcular una operación que combine tensores en diferentes dispositivos, ¡debemos moverlos primero! Podemos usar el método `.to()` como antes, o los métodos `.cpu()` y `.cuda()`. Ten en cuenta que el uso de `.cuda()` arrojará un error, si CUDA no está habilitado en tu máquina.

Generalmente, en este curso, todo el Deep Learning se hace en la GPU, y cualquier cálculo se hace en la CPU, así que a veces tenemos que pasar cosas de un lado a otro, por lo que nos verás llamar.

In [ ]:
x = torch.tensor([0, 1, 2], device=DEVICE)
y = torch.tensor([3, 4, 5], device="cpu")
z = torch.tensor([6, 7, 8], device=DEVICE)

# moving to cpu
x = x.to("cpu")  # alternatively, you can use x = x.cpu()
print(x + y)

# moving to gpu
y = y.to(DEVICE)  # alternatively, you can use y = y.cuda()
print(y + z)

### Ejercicio 2.4: ¿Cuánto más rápidas son las GPU?

A continuación se muestra una función simple `simpleFun`. Completa esta función, de forma que realice las operaciones

- Multiplicación por elementos

- Multiplicación de matrices

Las operaciones deben poder realizarse en la CPU o en la GPU especificadas por el parámetro `device`. Utilizaremos la función de ayuda `timeFun(f, dim, iteraciones, dispositivo)`.

In [37]:
dim = 10000
iterations = 1

In [ ]:
def simpleFun(dim, device):
  """
  Helper function to check device-compatiblity with computations

  Args:
    dim: Integer
    device: String
      "cpu" or "cuda"

  Returns:
    Nothing.
  """
  ###############################################
  ## TODO for students: recreate the function, but
  ## ensure all computations happens on the `device`
  raise NotImplementedError("Student exercise: fill in the missing code to create the tensors")
  ###############################################
  # 2D tensor filled with uniform random numbers in [0,1), dim x dim
  x = ...
  # 2D tensor filled with uniform random numbers in [0,1), dim x dim
  y = ...
  # 2D tensor filled with the scalar value 2, dim x dim
  z = ...

  # elementwise multiplication of x and y
  a = ...
  # matrix multiplication of x and z
  b = ...

  del x
  del y
  del z
  del a
  del b


## TODO: Implement the function above and uncomment the following lines to test your code
# timeFun(f=simpleFun, dim=dim, iterations=iterations)
# timeFun(f=simpleFun, dim=dim, iterations=iterations, device=DEVICE)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_6284a222.py)



Sample output (depends on your hardware)
```
time taken for 1 iterations of simpleFun(10000, cpu): 23.74070
time taken for 1 iterations of simpleFun(10000, cuda): 0.87535
```

**¡Discute!**

Intenta reducir las dimensiones de los tensores y aumentar las iteraciones. Puedes llegar a un punto en el que la función de sólo cpu sea más rápida que la de GPU. ¿Por qué puede ser esto?

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_4ee0dee3.py)



## Sección 2.5: Conjuntos de datos y cargadores de datos

Cuando se entrenan modelos de redes neuronales se trabaja con grandes cantidades de datos. Afortunadamente, PyTorch ofrece algunas herramientas estupendas que te ayudan a organizar y manipular tus muestras de datos.

In [ ]:
# Import dataset and dataloaders related packages
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Grayscale

**Conjuntos de datos**

El paquete `torchvision` le permite acceder fácilmente a muchos de los conjuntos de datos disponibles públicamente. Vamos a cargar el conjunto de datos [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html), que contiene imágenes en color de 10 clases diferentes, como vehículos y animales.

La creación de un objeto de tipo `datasets.CIFAR10` descargará y cargará automáticamente todas las imágenes del conjunto de datos. La estructura de datos resultante puede tratarse como una lista que contiene muestras de datos y sus correspondientes etiquetas.

In [ ]:
# Download and load the images from the CIFAR10 dataset
cifar10_data = datasets.CIFAR10(
    root="data",  # path where the images will be stored
    download=True,  # all images should be downloaded
    transform=ToTensor()  # transform the images to tensors
    )

# Print the number of samples in the loaded dataset
print(f"Number of samples: {len(cifar10_data)}")
print(f"Class names: {cifar10_data.classes}")

Tenemos 50.000 muestras cargadas. Ahora, vamos a ver una de ellas en detalle. Cada muestra consta de una imagen y su correspondiente etiqueta.

In [ ]:
# Choose a random sample
random.seed(2021)
image, label = cifar10_data[random.randint(0, len(cifar10_data))]
print(f"Label: {cifar10_data.classes[label]}")
print(f"Image size: {image.shape}")

Las imágenes en color se modelan como tensores tridimensionales. La primera dimensión corresponde a los canales ($\text{C}$) de la imagen (en este caso tenemos imágenes RGB). La segunda dimensión es la altura ($\text{H}$) de la imagen y la tercera es la anchura ($\text{W}$). Podemos denotar este formato de imagen como $\text{C} \times \text{H} \times \text{W}$.

### Ejercicio 2.5: Visualizar una imagen del conjunto de datos

Intentemos mostrar la imagen utilizando `matplotlib`. El código siguiente no funcionará, porque `imshow` espera tener la imagen en un formato diferente, es decir, $\text{C} \times \text{H} \times \text{W}$.

Es necesario reordenar las dimensiones del tensor utilizando el método `permute` del tensor. PyTorch `torch.permute(*dims)` reordena el tensor original según el ordenamiento deseado y devuelve un nuevo tensor multidimensional rotado. El tamaño del tensor devuelto sigue siendo el mismo que el del original.

**Code hint:**

```python
# create a tensor of size 2 x 4
input_var = torch.randn(2, 4)
# print its size and the tensor
print(input_var.size())
print(input_var)

# dimensions permuted
input_var = input_var.permute(1, 0)
# print its size and the permuted tensor
print(input_var.size())
print(input_var)
```

In [ ]:
# TODO: Uncomment the following line to see the error that arises from the current image format
# plt.imshow(image)

# TODO: Comment the above line and fix this code by reordering the tensor dimensions
# plt.imshow(image.permute(...))
# plt.show()

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_b04bd357.py)

*Example output:*

<img alt='Solution hint' align='left' width=835.0 height=827.0 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content-dl/main/tutorials/W1D1_BasicsAndPytorch/static/W1D1_Tutorial1_Solution_b04bd357_0.png>



**Conjuntos de datos de entrenamiento y de prueba**

Al cargar un conjunto de datos, puede especificar si desea cargar las muestras de entrenamiento o de prueba utilizando el argumento `train`. Podemos cargar los conjuntos de datos de entrenamiento y de prueba por separado. Por simplicidad, hoy no utilizaremos ambos conjuntos de datos por separado, pero este tema se tratará en los próximos días.

In [ ]:
# Load the training samples
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )

# Load the test samples
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
    )

**Cargador de datos**

Otro concepto importante es el `Dataloader`. Se trata de una envoltura alrededor del `Dataset` que lo divide en minilotes (importante para el entrenamiento de la red neuronal) y hace que los datos sean iterables. El argumento `shuffle` se utiliza para mezclar el orden de las muestras en los minilotes.

In [ ]:
# Create dataloaders with
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

*Reproducibilidad:* DataLoader resembrará a los trabajadores siguiendo el algoritmo de carga de datos multiproceso al azar. Utilice `worker_init_fn()` y un `generador` para preservar la reproducibilidad:


```python
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  numpy.random.seed(worker_seed)
  random.seed(worker_seed)


g_seed = torch.Generator()
g_seed.manual_seed(my_seed)

DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    worker_init_fn=seed_worker,
    generator=g_seed
    )
```

**Importante:** Para que el `seed_worker` tenga efecto, `num_workers` debe ser 2 o más.

Ahora podemos consultar el siguiente lote desde el cargador de datos e inspeccionarlo. Para ello tenemos que convertir el objeto del cargador de datos en un iterador de Python utilizando la función `iter` y luego podemos consultar el siguiente lote utilizando la función `next`.

Ahora podemos ver que tenemos un tensor 4D. Esto se debe a que tenemos 64 imágenes en el lote ($B$) y cada imagen tiene 3 dimensiones: canales ($C$), altura ($H$) y anchura ($W$). Por lo tanto, el tamaño del tensor 4D es $B \times C \times H \times W$.

In [ ]:
# Load the next batch
batch_images, batch_labels = next(iter(train_dataloader))
print('Batch size:', batch_images.shape)

# Display the first image from the batch
plt.imshow(batch_images[0].permute(1, 2, 0))
plt.show()

**Transformaciones**

Otra característica útil al cargar un conjunto de datos es la aplicación de transformaciones en los datos - conversiones de color, normalización, recorte, rotación, etc. Hay muchas transformaciones predefinidas en el paquete `torchvision.transforms` y también puedes combinarlas usando la transformación `Compose`. Consulta la [documentación de pytorch](https://pytorch.org/vision/stable/transforms.html) para más detalles.

### Ejercicio 2.6: Cargar el conjunto de datos CIFAR10 como imágenes en escala de grises

El objetivo de este ejercicio es cargar las imágenes del conjunto de datos CIFAR10 como imágenes en escala de grises. Tenga en cuenta que volvemos a ejecutar la función `set_seed` para garantizar la reproducibilidad.

In [ ]:
def my_data_load():
  """
  Function to load CIFAR10 data as grayscale images

  Args:
    None

  Returns:
    data: DataFrame
      CIFAR10 loaded Dataframe of shape (3309, 14)
  """
  ###############################################
  ## TODO for students: load the CIFAR10 data,
  ## but as grayscale images and not as RGB colored.
  raise NotImplementedError("Student exercise: fill in the missing code to load the data")
  ###############################################
  ## TODO Load the CIFAR10 data using a transform that converts the images to grayscale tensors
  data = datasets.CIFAR10(...,
                          transform=...)
  # Display a random grayscale image
  image, label = data[random.randint(0, len(data))]
  plt.imshow(image.squeeze(), cmap="gray")
  plt.show()

  return data


set_seed(seed=2021)
## After implementing the above code, uncomment the following lines to test your code
# data = my_data_load()

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_1c5a709e.py)

*Example output:*

<img alt='Solution hint' align='left' width=835.0 height=827.0 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content-dl/main/tutorials/W1D1_BasicsAndPytorch/static/W1D1_Tutorial1_Solution_1c5a709e_1.png>



---
# Sección 3: Redes neuronales

Ahora es el momento de crear tu primera red neuronal usando PyTorch. Esta sección te guiará a través del proceso de:

- Crear un modelo de red neuronal simple
- Entrenar la red
- Visualizar los resultados de la red
- Ajustar la red

## Sección 3.1: Carga de datos

Primero necesitamos algunos datos de ejemplo para entrenar nuestra red. Puedes utilizar la función de abajo para generar un conjunto de datos de ejemplo que consiste en puntos 2D a lo largo de dos medios círculos intercalados. Los datos se almacenarán en un archivo llamado `sample_data.csv`. Puede inspeccionar el archivo directamente en Colab yendo a Archivos en el lado izquierdo y abriendo el archivo CSV.

In [11]:
# @title Generate sample data
# @markdown we used `scikit-learn` module
from sklearn.datasets import make_moons

# Create a dataset of 256 points with a little noise
X, y = make_moons(256, noise=0.1)

# Store the data as a Pandas data frame and save it to a CSV file
df = pd.DataFrame(dict(x0=X[:,0], x1=X[:,1], y=y))
df.to_csv('sample_data.csv')

Ahora podemos cargar los datos del archivo CSV utilizando la biblioteca Pandas. Pandas proporciona muchas funciones para leer archivos en varios formatos. Al cargar los datos de un archivo CSV, podemos hacer referencia a las columnas directamente por sus nombres.

In [ ]:
# Load the data from the CSV file in a Pandas DataFrame
data = pd.read_csv("sample_data.csv")

# Create a 2D numpy array from the x0 and x1 columns
X_orig = data[["x0", "x1"]].to_numpy()

# Create a 1D numpy array from the y column
y_orig = data["y"].to_numpy()

# Print the sizes of the generated 2D points X and the corresponding labels Y
print(f"Size X:{X_orig.shape}")
print(f"Size y:{y_orig.shape}")

# Visualize the dataset. The color of the points is determined by the labels `y_orig`.
plt.scatter(X_orig[:, 0], X_orig[:, 1], s=40, c=y_orig)
plt.show()

**Preparar los datos para PyTorch**

Ahora vamos a preparar los datos en un formato adecuado para PyTorch - convertir todo en tensores.

In [ ]:
# Initialize the device variable
DEVICE = set_device()

# Convert the 2D points to a float32 tensor
X = torch.tensor(X_orig, dtype=torch.float32)

# Upload the tensor to the device
X = X.to(DEVICE)

print(f"Size X:{X.shape}")

# Convert the labels to a long interger tensor
y = torch.from_numpy(y_orig).type(torch.LongTensor)

# Upload the tensor to the device
y = y.to(DEVICE)

print(f"Size y:{y.shape}")

## Sección 3.2: Crear una red neuronal simple

Para este ejemplo queremos tener una red neuronal simple que conste de 3 capas:

- 1 capa de entrada de tamaño 2 (nuestros puntos tienen 2 coordenadas)
- 1 capa oculta de tamaño 16 (se puede jugar con diferentes números aquí)
- 1 capa de salida de tamaño 2 (queremos tener las puntuaciones de las dos clases)

A lo largo del curso te enfrentarás a diferentes tipos de redes neuronales. El ejemplo que se presenta aquí pretende demostrar el proceso de creación y entrenamiento de una red neuronal de principio a fin.

**Programando la red**

PyTorch proporciona una clase base para todos los módulos de redes neuronales llamada [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). Es necesario heredar de `nn.Module` e implementar algunos métodos importantes:

* `__init__`

  En el método `__init__` necesitas definir la estructura de tu red. Aquí especificarás de qué capas constará la red, qué funciones de activación se utilizarán, etc.

* `forward`

  Todos los módulos de redes neuronales necesitan implementar el método `forward`. Especifica los cálculos que la red necesita hacer cuando los datos pasan por ella.

* `predict`

  Este no es un método obligatorio de un módulo de red neuronal, pero es una buena práctica si quieres obtener rápidamente la etiqueta más probable de la red. Llama al método `forward` y elige la etiqueta con la mayor puntuación.

* `train` (entrenar)

  Este tampoco es un método obligatorio, pero es una buena práctica a tener en cuenta. The method will be used to train the network parameters and will be implemented later in the notebook.

<br>

**Nota:** Puedes usar el método `__call__` de un módulo directamente y éste invocará el método `forward`: `net()` hace lo mismo que `net.forward()`.

In [16]:
# Inherit from nn.Module - the base class for neural network modules provided by Pytorch
class NaiveNet(nn.Module):
  """
  NaiveNet architecture
  Structure is as follows:
  Linear Layer (2, 16) -> ReLU activation -> Linear Layer (16, 2)
  """
  # Define the structure of your network
  def __init__(self):
    """
    Defines the NaiveNet structure by initialising following attributes
    nn.Linear (2, 16):  Transformation from the input to the hidden layer
    nn.ReLU: Activation function (ReLU) is a non-linearity which is widely used because it reduces computation.
             The function returns 0 if it receives any negative input, but for any positive value x, it returns that value back.
    nn.Linear (16, 2): Transformation from the hidden to the output layer

    Args:
      None

    Returns:
      Nothing
    """
    super(NaiveNet, self).__init__()

    # The network is defined as a sequence of operations
    self.layers = nn.Sequential(
        nn.Linear(2, 16),
        nn.ReLU(),
        nn.Linear(16, 2),
    )

  # Specify the computations performed on the data
  def forward(self, x):
    """
    Defines the forward pass through the above defined structure

    Args:
      x: torch.Tensor
        Input tensor of size ([3])

    Returns:
      layers: nn.module
        Initialised Layers in order to re-use the same layer for each forward pass of data you make.
    """
    # Pass the data through the layers
    return self.layers(x)

  # Choose the most likely label predicted by the network
  def predict(self, x):
    """
    Performs the prediction task of the network

    Args:
      x: torch.Tensor
        Input tensor of size ([3])

    Returns:
      Most likely class i.e., Label with the highest score
    """
    # Pass the data through the networks
    output = self.forward(x)

    # Choose the label with the highest score
    return torch.argmax(output, 1)

  # Train the neural network (will be implemented later)
  def train(self, X, y):
    """
    Training the Neural Network

    Args:
      X: torch.Tensor
        Input data
      y: torch.Tensor
        Class Labels/Targets

    Returns:
      Nothing
    """
    pass

**Comprueba que tu red funciona**

Crea una instancia de tu modelo y visualízala.

In [ ]:
# Create new NaiveNet and transfer it to the device
model = NaiveNet().to(DEVICE)

# Print the structure of the network
print(model)

### Ejercicio 3.2: Clasificar algunas muestras

Ahora, vamos a pasar algunos de los puntos de nuestro conjunto de datos a través de la red y ver si funciona. No hay que esperar que la red clasifique los puntos correctamente, porque aún no ha sido entrenada. 

El objetivo aquí es sólo obtener algo de experiencia con las estructuras de datos que se pasan a los métodos de avance y predicción y sus resultados.

In [ ]:
## Get the samples
# X_samples = ...
# print("Sample input:\n", X_samples)

## Do a forward pass of the network
# output = ...
# print("\nNetwork output:\n", output)

## Predict the label of each point
# y_predicted = ...
# print("\nPredicted labels:\n", y_predicted)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_63f6a21a.py)



```
Sample input:
 tensor([[ 0.9066,  0.5052],
        [-0.2024,  1.1226],
        [ 1.0685,  0.2809],
        [ 0.6720,  0.5097],
        [ 0.8548,  0.5122]], device='cuda:0')

Network output:
 tensor([[ 0.1543, -0.8018],
        [ 2.2077, -2.9859],
        [-0.5745, -0.0195],
        [ 0.1924, -0.8367],
        [ 0.1818, -0.8301]], device='cuda:0', grad_fn=<AddmmBackward>)

Predicted labels:
 tensor([0, 0, 1, 0, 0], device='cuda:0')
```

## Sección 3.3: Entrenar la red neuronal

Ahora es el momento de entrenar tu red en tu conjunto de datos. No te preocupes si aún no lo entiendes del todo: en las próximas clases trataremos el entrenamiento con mucho más detalle. Por ahora, el objetivo es ver tu red en acción.

Normalmente implementarás el método `train` directamente cuando implementes tu clase `NaiveNet`. Aquí, lo implementaremos como una función fuera de la clase para tenerlo en una celda separada.

In [18]:
# @title Función de ayuda para trazar el límite de decisión

# Code adapted from this notebook: https://jonchar.net/notebooks/Artificial-Neural-Network-with-Keras/

from pathlib import Path

def plot_decision_boundary(model, X, y, device):
  """
  Helper function to plot decision boundary

  Args:
    model: nn.module
      NaiveNet instance
    X: torch.tensor
      Input CIFAR10 data
    y: torch.tensor
      Class Labels/Targets
    device: String
      "cpu" or "cuda"

  Returns:
    Nothing
  """
  # Transfer the data to the CPU
  X = X.cpu().numpy()
  y = y.cpu().numpy()

  # Check if the frames folder exists and create it if needed
  frames_path = Path("frames")
  if not frames_path.exists():
    frames_path.mkdir()

  # Set min and max values and give it some padding
  x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
  y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
  h = 0.01

  # Generate a grid of points with distance h between them
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

  # Predict the function value for the whole gid
  grid_points = np.c_[xx.ravel(), yy.ravel()]
  grid_points = torch.from_numpy(grid_points).type(torch.FloatTensor)
  Z = model.predict(grid_points.to(device)).cpu().numpy()
  Z = Z.reshape(xx.shape)

  # Plot the contour and training examples
  plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.binary)

In [ ]:
# Implement the train function given a training dataset X and correcsponding labels y
def train(model, X, y):
  """
    Training the Neural Network

    Args:
      X: torch.Tensor
        Input data
      y: torch.Tensor
        Class Labels/Targets

    Returns:
      losses: Float
        Cross Entropy Loss; Cross-entropy builds upon the idea of entropy
        from information theory and calculates the number of bits required
        to represent or transmit an average event from one distribution
        compared to another distribution.
    """
  # The Cross Entropy Loss is suitable for classification problems
  loss_function = nn.CrossEntropyLoss()

  # Create an optimizer (Stochastic Gradient Descent) that will be used to train the network
  learning_rate = 1e-2
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

  # Number of epochs
  epochs = 15000

  # List of losses for visualization
  losses = []

  for i in range(epochs):
    # Pass the data through the network and compute the loss
    # We'll use the whole dataset during the training instead of using batches
    # in to order to keep the code simple for now.
    y_logits = model.forward(X)
    loss = loss_function(y_logits, y)

    # Clear the previous gradients and compute the new ones
    optimizer.zero_grad()
    loss.backward()

    # Adapt the weights of the network
    optimizer.step()

    # Store the loss
    losses.append(loss.item())

    # Print the results at every 1000th epoch
    if i % 1000 == 0:
      print(f"Epoch {i} loss is {loss.item()}")

      plot_decision_boundary(model, X, y, DEVICE)
      plt.savefig('frames/{:05d}.png'.format(i))

  return losses


# Create a new network instance a train it
model = NaiveNet().to(DEVICE)
losses = train(model, X, y)

**Graficar la pérdida durante el entrenamiento**

Grafique la pérdida durante el entrenamiento para ver cómo se reduce y converge.

In [ ]:
plt.plot(np.linspace(1, len(losses), len(losses)), losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")

In [ ]:
# @title Visualize the training process
# @markdown Execute this cell!
!pip install imageio --quiet
!pip install pathlib --quiet

import imageio
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image, display
from pathlib import Path

InteractiveShell.ast_node_interactivity = "all"

# Make a list with all images
images = []
for i in range(10):
  filename = Path("frames/0"+str(i)+"000.png")
  images.append(imageio.imread(filename))
# Save the gif
imageio.mimsave('frames/movie.gif', images)
gifPath = Path("frames/movie.gif")
with open(gifPath,'rb') as f:
  display(Image(data=f.read(), format='png'))

### Ejercicio 3.3: Ajuste su red

Ahora puedes jugar un poco con la red para tener una idea de lo que hacen los diferentes parámetros. Aquí tienes algunas ideas que puedes probar:

- Aumentar o disminuir el número de épocas de entrenamiento
- Aumentar o disminuir el tamaño de la capa oculta
- Añadir una capa oculta adicional

¿Puedes conseguir que la red se ajuste mejor a los datos?

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_fd3bd4a1.py)



La operación lógica OR exclusiva (XOR) da una salida verdadera (`1`) cuando el número de entradas verdaderas es impar. Es decir, una salida verdadera si una, y sólo una, de las entradas de la puerta es verdadera. Si ambas entradas son falsas (`0`) o ambas son verdaderas, la salida es falsa. Matemáticamente, XOR representa la función de desigualdad, es decir, la salida es verdadera si las entradas no son iguales; en caso contrario, la salida es falsa.

En el caso de dos entradas ($X$ y $Y$) se aplica la siguiente tabla de verdad:

\begin{matrix}
  X & Y & \text{XOR}\\
  \hline
  0 & 0 & 0\\
  0 & 1 & 1\\
  1 & 0 & 1\\
  1 & 1 & 0
\end{matrix}

Aquí, con `0`, denotamos `Falso`, y con `1` denotamos `Verdadero` en términos booleanos.

### Demostración interactiva 3.3: Resolución de XOR

Aquí utilizamos un famoso widget de visualización de código abierto desarrollado por el equipo de Tensorflow disponible [aquí](https://github.com/tensorflow/playground).
* Juega con el widget y observa si puedes resolver el conjunto de datos XOR continuo.
* Ahora añade una capa oculta con tres unidades, juega con el widget y establece los pesos a mano para resolver este conjunto de datos perfectamente.

Para la segunda parte, debes establecer los pesos haciendo clic en las conexiones y escribir el valor o utilizar las teclas arriba y abajo para cambiarlo en un incremento. También puedes hacer lo mismo con los sesgos haciendo clic en el pequeño cuadrado de la parte inferior izquierda de cada neurona.
A pesar de que hay infinitas soluciones, una solución limpia cuando $f(x)$ es ReLU es: 

\begin{equation}
  y = f(x_1)+f(x_2)-f(x_1+x_2)
\end{equation}

Trate de establecer los pesos y los sesgos para implementar esta función después de jugar lo suficiente :)

In [ ]:
# @markdown Play with the parameters to solve XOR
from IPython.display import IFrame
IFrame("https://playground.arashash.com/#activation=relu&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=&seed=0.91390&showTestData=false&discretize=false&percTrainData=90&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false", width=1020, height=660)

In [ ]:
# @markdown Do you think we can solve the discrete XOR (only 4 possibilities) with only 2 hidden units?
w1_min_xor = 'No' #@param ['Select', 'Yes', 'No']
if w1_min_xor == 'No':
  print("Correct!")
else:
  print("How about giving it another try?")

---
# Bonus - 60 años de investigación sobre aprendizaje automático en una parcela

Por [Hendrik Strobelt](http://hendrik.strobelt.com) (MIT-IBM Watson AI Lab) con el apoyo de Benjamin Hoover.

En este cuaderno visualizamos un subconjunto* de 3.300 artículos extraídos del conjunto de datos AllenAI [S2ORC](https://github.com/allenai/s2orc). Representamos cada artículo mediante una posición que es el resultado de un método de reducción de la dimensionalidad aplicado a una representación vectorial de cada artículo. La representación vectorial es la salida de una red neuronal.

**Nota:** La selección está muy sesgada por las palabras clave y la metodología que utilizamos para filtrar. Por favor, consulte la sección de detalles para conocer lo que hicimos. 

In [28]:
# @title Install and Import `altair` and `vega_datasets`.
!pip install altair vega_datasets --quiet

import altair as alt  # altair is defining data visualizations

# Source data files
# Position data file maps ID to x,y positions
# original link: http://gltr.io/temp/ml_regexv1_cs_ma_citation+_99perc.pos_umap_cosine_100_d0.1.json
POS_FILE = 'https://osf.io/qyrfn/download'
# original link: http://gltr.io/temp/ml_regexv1_cs_ma_citation+_99perc_clean.csv
# Metadata file maps ID to title, abstract, author,....
META_FILE = 'https://osf.io/vfdu6/download'

# data loading and wrangling
def load_data():
  """
  Loading the data

  Args:
    None

  Returns:
    Merged read dataFrame combining id and paper_id;
  """
  positions = pd.read_json(POS_FILE)
  positions[['x', 'y']] = positions['pos'].to_list()
  meta = pd.read_csv(META_FILE)
  return positions.merge(meta, left_on='id', right_on='paper_id')


# load data
data = load_data()

In [29]:
# @title Define Visualization using ALtair
YEAR_PERIOD = "quinquennial"  # @param
selection = alt.selection_multi(fields=[YEAR_PERIOD], bind='legend')
data[YEAR_PERIOD] = (data["year"] / 5.0).apply(np.floor) * 5
chart = alt.Chart(data[["x", "y", "authors", "title", YEAR_PERIOD, "citation_count"]], width=800,
                  height=800).mark_circle(radius=2, opacity=0.2).encode(
    alt.Color(YEAR_PERIOD+':O',
              scale=alt.Scale(scheme='viridis', reverse=False, clamp=True, domain=list(range(1955,2020,5))),
              # legend=alt.Legend(title='Total Records')
              ),
    alt.Size('citation_count',
              scale=alt.Scale(type="pow", exponent=1, range=[15, 300])
              ),
       alt.X('x:Q',
        scale=alt.Scale(zero=False), axis=alt.Axis(labels=False)
    ),
       alt.Y('y:Q',
        scale=alt.Scale(zero=False), axis=alt.Axis(labels=False)
    ),
    tooltip=['title', 'authors'],
    # size='citation_count',
    # color="decade:O",
    opacity=alt.condition(selection, alt.value(.8), alt.value(0.2)),

).add_selection(
    selection
).interactive()

Veamos la visualización. Cada punto representa un artículo. Los puntos cercanos significan que los artículos respectivos están más relacionados que los distantes. El color indica el período de 5 años en que se publicó el artículo. El tamaño del punto indica el número de citas (dentro del corpus S2ORC) en julio de 2020. 

La vista es **interactiva** y permite tres interacciones principales. Pruébelas y juegue con ellas:
1. Pasa el ratón por encima de un punto para ver un tooltip (título, autor)
2. Seleccione un año en la leyenda (derecha) para filtrar los puntos
3. Acercarse o alejarse con el scroll -- un doble clic restablece la vista

In [ ]:
chart

## Preguntas

Jugando, ¿puedes encontrar algunas respuestas a las siguientes preguntas?

1. ¿Puedes encontrar clusters temáticos? ¿Qué clúster podría producirse debido a un error de filtrado?
2. ¿Puedes ver una tendencia temporal en los datos y los clusters?
3. ¿Puede determinar cuándo empezaron a estar en auge los métodos de aprendizaje profundo?
4. ¿Puedes encontrar los artículos clave que se escribieron antes del "invierno" del DL y que definen los hitos de un clúster? (consejo: busque puntos grandes de diferente color)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content-dl/tree/main//tutorials/W1D1_BasicsAndPytorch/solutions/W1D1_Tutorial1_Solution_21a88cd0.py)



## Métodos

Esto es lo que hicimos:
1. Filtrado de todos los artículos que cumplían los criterios
  - se clasifican como `Ciencias de la Computación` o `Matemáticas` 
  - que aparezca una de las siguientes palabras clave en el título o en el resumen: `"machine learning|artificial intelligence|neural network|(machine|computer) vision|perceptron|network architecture| RNN | CNN | LSTM | BLEU | MNIST | CIFAR |reinforcement learning|gradient descent| Imagenet"`.
2. Por año, elimine todos los artículos que estén por debajo del percentil 99 del recuento de citas en ese año
3. Incorporar cada artículo utilizando el resumen + el título en el modelo SPECTER
4. Proyecto basado en la incrustación utilizando UMAP
5. Visualizar con Altair

### Buscar autores

In [34]:
# @title Edit the `AUTHOR_FILTER` variable to full text search for authors.

AUTHOR_FILTER = "Hinton"  # @param space at the end means "word border"

### Don't ignore case when searching...
FLAGS = 0
### uncomment do ignore case
# FLAGS = re.IGNORECASE

## --- FILTER CODE.. make it your own ---
data['issel'] = data['authors'].str.contains(AUTHOR_FILTER, na=False, flags=FLAGS, )
if data['issel'].mean()<0.0000000001:
  print('No match found')

## --- FROM HERE ON VIS CODE ---
alt.Chart(data[["x", "y", "authors", "title", YEAR_PERIOD, "citation_count", "issel"]], width=800,
                  height=800) \
    .mark_circle(stroke="black", strokeOpacity=1).encode(
    alt.Color(YEAR_PERIOD+':O',
              scale=alt.Scale(scheme='viridis', reverse=False),
              # legend=alt.Legend(title='Total Records')
              ),
    alt.Size('citation_count',
              scale=alt.Scale(type="pow", exponent=1, range=[15, 300])
              ),
    alt.StrokeWidth('issel:Q', scale=alt.Scale(type="linear", domain=[0,1], range=[0, 2]), legend=None),

    alt.Opacity('issel:Q', scale=alt.Scale(type="linear", domain=[0,1], range=[.2, 1]), legend=None),
    alt.X('x:Q',
        scale=alt.Scale(zero=False), axis=alt.Axis(labels=False)
    ),
    alt.Y('y:Q',
        scale=alt.Scale(zero=False), axis=alt.Axis(labels=False)
    ),
    tooltip=['title', 'authors'],
).interactive()

alt.Chart(...)

---
# Appendix

## Official PyTorch resources:

### Tutorials
- [https://pytorch.org/tutorials/](https://pytorch.org/tutorials/)

### Documentation
- [https://pytorch.org/docs/stable/tensors.html](https://pytorch.org/docs/stable/tensors.html) (tensor methods)

- [https://pytorch.org/docs/stable/generated/torch.Tensor.view.html#torch.Tensor.view](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html#torch.Tensor.view) (The view method in particular)

- [https://pytorch.org/vision/stable/datasets.html](https://pytorch.org/vision/stable/datasets.html) (pre-loaded image datasets)

## Google Colab Resources:
- [https://research.google.com/colaboratory/faq.html](https://research.google.com/colaboratory/faq.html) (FAQ including guidance on GPU usage)

## Books for reference:
- [https://www.deeplearningbook.org/](https://www.deeplearningbook.org/) (Deep Learning by Ian Goodfellow, Yoshua Bengio and Aaron Courville)